# Internet Computer ![ICP logo](https://cdn-assets-eu.frontify.com/s3/frontify-enterprise-files-eu/eyJwYXRoIjoiZGZpbml0eVwvZmlsZVwvQ3ZFa3Y5eFhQOGQ3dW83ZkxFMmkucG5nIn0:dfinity:WGPFvOEBt8LexZ0568duC01k5g99_6IN-Ju8n3WvJhE?width=115&format=webp&quality=100)

## Setup 🛠️
* Install [Rust](https://www.rust-lang.org/tools/install)
* Install [dfx](https://internetcomputer.org/docs/building-apps/getting-started/install#installing-dfx-via-dfxvm)
* Checkout the `basic_solana` example
    ```
    git clone git@github.com:dfinity/sol-rpc-canister.git
    cd sol-rpc-canister/examples/basic_solana
    ```

In [ ]:
!dfx --version

In [ ]:
!pwd

### Deploy

* Deploy `basic_solana` example on mainnet.


![Setup](./helium_demo_setup.png)

In [ ]:
!dfx identity use admin

In [ ]:
!dfx deploy --ic basic_solana --argument '(\
    record {\
        solana_commitment_level = opt variant { confirmed };\
        ed25519_key_name = opt variant { MainnetTestKey1 };\
        solana_network = opt variant { Devnet };\
        sol_rpc_canister_id = opt principal "titvo-eiaaa-aaaar-qaogq-cai";\
        }\
)'

In [ ]:
!dfx canister status --ic --all

### Create a new identity
Create a new identity, which is not a controller of the `basic_solana` canister, to interact with it.

In [ ]:
!dfx identity new helium_demo

In [ ]:
!dfx identity use helium_demo

In [ ]:
!dfx identity get-principal

In [ ]:
%env MY_PRINCIPAL="2oyh2-miczk-rzcqm-zbkes-q3kyi-lmen7-slvvl-byown-zz6v6-razzx-vae"

## Read Balance 🔎

In [ ]:
%env SOLANA_CLI_ADDRESS="AhwdjZNRkqXeRfujdBnVqC8oe9kdpDKDM6efioKq1dBZ"

In [ ]:
!dfx canister call basic_solana get_balance "opt (${SOLANA_CLI_ADDRESS})" --ic

### ℹ️ Explanations

* SOL has 9 decimals: 1 SOL == `1_000_000_000` Lamports
* [Logs of SOL RPC canister](https://titvo-eiaaa-aaaar-qaogq-cai.raw.icp0.io/logs?sort=desc)
    * 3 [HTTPS outcalls](https://support.dfinity.org/hc/en-us/articles/12342341722260-What-is-an-HTTPS-Outcall):
        * `solana-devnet.g.alchemy.com`
        * `lb.drpc.org`
        * `devnet.helius-rpc.com`
    * each HTTPS outcall involves each replica in the subnet making an HTTPS request => 34 HTTPs request for the [fiduciary subnet](https://dashboard.internetcomputer.org/subnet/pzp6e-ekpqk-3c5x7-2h6so-njoeq-mt45d-h3h6c-q3mxf-vpeq5-fk5o7-yae)
    *  TOTAL: 102 HTTPS requests distributed all-over the world! 💪
    * **no single point of failure** 🛡️



*Request airdrop with Solana CLI*


In [ ]:
!dfx canister call basic_solana get_balance "opt (${SOLANA_CLI_ADDRESS})" --ic


## Send SOL 🪙

ℹ️ Canister can control an Ed25519 public/private key pair thanks to [threshold Schnorr](https://internetcomputer.org/docs/building-apps/network-features/signatures/t-schnorr/):
* Each replica holds a share of a secret key
* Each replica can use its share to create a signature share for a given message
* Combining enough signature shares result in a standard signature for that message
* At no moment is the secret key reconstructed.

### Fund address

In [ ]:
!dfx canister call basic_solana solana_account --ic

In [ ]:
%env SOLANA_CANISTER_ADDRESS="8oU5d6pUjcx9iVMh3yEDJTH4ghdbtNUAkr8rn6s68TGc"

In [ ]:
!dfx canister call basic_solana get_balance --ic

*Fund address via Solana CLI*

In [ ]:
!dfx canister call basic_solana get_balance --ic


### Send transaction

Send transaction of 1 Lamport back to Solana CLI address

In [ ]:
!dfx canister call basic_solana send_sol "(opt principal $MY_PRINCIPAL, $SOLANA_CLI_ADDRESS, 1 : nat)" --ic

Check transaction on [Solana explorer](https://explorer.solana.com/?cluster=devnet):
* nothing distinguishes a transaction signed via threshold Ed25529 from a regular transaction
* address controlled by the canister also paid the transaction fee

In [ ]:
!dfx canister call basic_solana get_balance --ic


* 1 major difficulty is getting a recent blockhash that must be part of the transaction.
* See the [code](https://github.com/dfinity/sol-rpc-canister/blob/8e7e32f50a2e05fa671a3df8f7b19f64c5abbdce/examples/basic_solana/src/main.rs#L226) for more details.

## Send USDC 💰

Send 1 USDC from addressed controlled by canister to my Solana CLI wallet
* USDC devnet: [4zMMC9srt5Ri5X14GAgXhaHii3GnPAEERYPJgZJDncDU](https://explorer.solana.com/address/4zMMC9srt5Ri5X14GAgXhaHii3GnPAEERYPJgZJDncDU?cluster=devnet)


In [ ]:
%env USDC_DEVNET="4zMMC9srt5Ri5X14GAgXhaHii3GnPAEERYPJgZJDncDU"

### Create associated token account

ℹ An associated token account is:
* needed to store the user's balance of a token => must be created before sending the first token.
* an address derived from a Solana account and a token account => different tokens, different associated token accounts.

In [ ]:
!dfx canister call basic_solana create_associated_token_account "(opt principal $MY_PRINCIPAL, $USDC_DEVNET)" --ic

### Fund address associated token account

* USDC [faucet](https://faucet.circle.com/)


In [ ]:
!dfx canister call basic_solana get_spl_token_balance "(null, $USDC_DEVNET)" --ic


### Send transaction

In [ ]:
!dfx canister call basic_solana send_spl_token "(opt principal $MY_PRINCIPAL, $USDC_DEVNET, $SOLANA_CLI_ADDRESS, 1_000_000 : nat)" --ic

In [ ]:
!dfx canister call basic_solana get_spl_token_balance "(null, $USDC_DEVNET)" --ic
